In [ ]:
import pandas as pd 
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras import layers
import os
from sklearn.metrics import classification_report,  confusion_matrix, accuracy_score, f1_score,  precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler


random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Loading and Prepping data

## Train Set

In [ ]:
train_data = {}
for year in range(2010,2017):
  train_data[year] = pd.read_csv(f'https://raw.githubusercontent.com/jacobh310/over_under/master/data_cleaning/clean_data/clean_data_{year}.csv')

dev_data = {}
for year in range(2017,2019):
  dev_data[year] = pd.read_csv(f'https://raw.githubusercontent.com/jacobh310/over_under/master/data_cleaning/clean_data/clean_data_{year}.csv')

In [ ]:
def prep_data(train_data, dev_data, scale):  
  
  train = pd.concat([train_data[year] for year in range(2010,2017)])
  train = train.dropna()

  train_features = train.drop(columns=['Date','Home Team','Visit Team','Total Runs','Over']).values #features
  
  train_labels = train['Over'].values  # labels

  dev = pd.concat([dev_data[year] for year in range(2017,2019)])
  dev_features = dev.drop(columns=['Date','Home Team','Visit Team','Total Runs','Over']).values # features
  dev_labels = dev['Over'].values # lables

  if scale:
    train_scale = MinMaxScaler()
    dev_scale = MinMaxScaler()

    train_features = train_scale.fit_transform(train_features)    # scale
    dev_features = dev_scale.fit_transform(dev_features) # scale

  return train_features, train_labels, dev_features, dev_labels




In [ ]:
train_features, train_labels, dev_features, dev_labels = prep_data(train_data, dev_data, False)

In [ ]:
BATCH_SIZE=128

train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_features,
    train_labels)
)

dev_dataset = tf.data.Dataset.from_tensor_slices(
    (dev_features,
    dev_labels)
)

train_dataset = train_dataset.batch(batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
dev_dataset = dev_dataset.batch(batch_size=BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


# Establishing Baseline

In [ ]:
train['Over'].value_counts(normalize=True)

1    0.520742
0    0.479258
Name: Over, dtype: float64

In [ ]:
dev['Over'].value_counts(normalize=True)

1    0.506729
0    0.493271
Name: Over, dtype: float64

# Model Building

## Machine Learning Models

In [ ]:
models = {'KNN': KNeighborsClassifier(n_jobs=-1),
          'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
          'XGBoost' : XGBClassifier(n_jobs=-1, random_state=42),
          'Naive Bayes Gaussian': GaussianNB(),
          'SVC': SVC(random_state=42, probability=True)
          }

results = {}

for model_name, model in models.items():
  model.fit(train_features, train_labels)
  results[model_name] = [model.score(train_features, train_labels), model.score(dev_features, dev_labels)]

In [ ]:
base_train_score = train['Over'].value_counts(normalize=True)[1]
base_dev_score = dev['Over'].value_counts(normalize=True)[1]
print('Baseline')
print(f"Train accuracy score: {base_train_score*100}")
print(f"Dev accuracy score: {base_dev_score*100}\n")

results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Train Accuracy Score','Dev Accuracy Score'])
results_df['Dev Improvement'] = base_dev_score
results_df['Dev Improvement'] = results_df['Dev Accuracy Score'] - results_df['Dev Improvement'] 
results_df = results_df*100
results_df

Baseline
Train accuracy score: 52.07419898819562
Dev accuracy score: 50.67294751009421



,Train Accuracy Score,Dev Accuracy Score,Dev Improvement
KNN,69.504216,50.134590,-0.538358
RandomForest,100.000000,51.323463,0.650516
XGBoost,60.924115,51.166442,0.493495
Naive Bayes Gaussian,50.940978,49.820547,-0.852400
SVC,52.121417,50.740242,0.067295


### Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV 

#### SVC

In [ ]:
svc = SVC(probability = True)
param_grid = [{'kernel': ['rbf'], 'gamma': [.1,.5,1,2,5,10], 'C': [.1, 1, 10, 100, 1000]},
              {'kernel': ['linear'], 'C': [.1, 1, 10, 100, 1000]},
              {'kernel': ['poly'], 'degree' : [2,3,4,5], 'C': [.1, 1, 10, 100, 1000]}]

clf_svc = GridSearchCV(svc, param_grid = param_grid, cv = 3, verbose = True, n_jobs = -1)
svc_grid = clf_svc.fit(train_features, train_labels)

Fitting 3 folds for each of 55 candidates, totalling 165 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 23.2min
[Parallel(n_jobs=-1)]: Done 165 out of 165 | elapsed: 455.0min finished


In [130]:
svc_best_params = {'C': 0.1, 'kernel': 'linear'}

In [131]:
svc_grid.best_estimator_.score(dev_features, dev_labels)

0.5067294751009421

#### RandomForest

## Neural Networks

In [ ]:
 METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')] 

In [ ]:
model = tf.keras.Sequential([
                             layers.Dense(21, activation='relu',input_shape=(21,)),
                             layers.Dropout(0.2),
                             layers.Dense(32, activation='relu'),
                             layers.Dropout(0.2),
                             layers.Dense(64, activation='relu'),
                             layers.Dropout(0.2),
                             layers.Dense(32, activation='relu'),
                             layers.Dropout(0.2),
                             layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=METRICS)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 21)                462       
_________________________________________________________________
dropout_8 (Dropout)          (None, 21)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 32)                704       
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)               

In [ ]:
model.fit(train_dataset,
          epochs=50,
          steps_per_epoch=len(train_dataset),
          validation_data = dev_dataset,
          validation_steps= len(dev_dataset),)

Epoch 1/50
116/116 [==============================] - 2s 10ms/step - loss: 0.6993 - accuracy: 0.5028 - precision: 0.5183 - recall: 0.6396 - val_loss: 0.6958 - val_accuracy: 0.5067 - val_precision: 0.5067 - val_recall: 0.9996
Epoch 2/50
116/116 [==============================] - 1s 6ms/step - loss: 0.6933 - accuracy: 0.5181 - precision: 0.5266 - recall: 0.7381 - val_loss: 0.6948 - val_accuracy: 0.5067 - val_precision: 0.5067 - val_recall: 1.0000
Epoch 3/50
116/116 [==============================] - 1s 6ms/step - loss: 0.6940 - accuracy: 0.5141 - precision: 0.5211 - recall: 0.8255 - val_loss: 0.6963 - val_accuracy: 0.5067 - val_precision: 0.5067 - val_recall: 1.0000
Epoch 4/50
116/116 [==============================] - 1s 6ms/step - loss: 0.6933 - accuracy: 0.5168 - precision: 0.5202 - recall: 0.9289 - val_loss: 0.6947 - val_accuracy: 0.5067 - val_precision: 0.5067 - val_recall: 1.0000
Epoch 5/50
116/116 [==============================] - 1s 6ms/step - loss: 0.6927 - accuracy: 0.5174 - p

In [ ]:
model.evaluate(dev_dataset)

35/35 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.5076 - precision: 0.5072 - recall: 0.9996


[0.6931648254394531,
 0.5076267123222351,
 0.5071877837181091,
 0.9995573163032532]